<a href="https://colab.research.google.com/github/yujinhor/ML_emergency-stop-prediction-pybullet/blob/main/accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive

# 1. 구글 드라이브 연결
if not os.path.exists('/content/drive'):
    print("📂 구글 드라이브 연결 중...")
    drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/Robot_Analysis'

# 파일 경로
PRED_PATH = os.path.join(BASE_DIR, 'danger_max_duration_summary.csv')  # AI 예측 파일
LABEL_PATH = os.path.join(BASE_DIR, 'val_summary.csv')                 # 실제 정답 파일
OUTPUT_PATH = os.path.join(BASE_DIR, 'simple_match_result.csv')        # 결과 저장

def main():
    print("📖 데이터 읽는 중...")

    # 데이터 로드
    df_pred = pd.read_csv(PRED_PATH)
    df_label = pd.read_csv(LABEL_PATH)

    # 2. 딱 필요한 정보만 뽑아서 병합 (Episode_ID 기준)
    # AI 파일에는 'Episode_ID', 정답 파일에는 'episode'로 되어 있을 수 있음

    # 정답 파일에서 에피소드 번호와 결과(result_is_failure)만 가져옴
    # (result_is_failure: 1=고장, 0=성공)
    label_subset = df_label[['episode', 'result_is_failure']].copy()
    label_subset.rename(columns={'episode': 'Episode_ID'}, inplace=True)

    # 병합 (Left Join: AI가 위험하다고 한 목록 기준)
    merged_df = pd.merge(df_pred[['Episode_ID', 'Avg_Prob']], label_subset, on='Episode_ID', how='left')

    # 3. 결과 판정 (O/X 채점)
    # AI는 이미 위험하다고 예측했으므로, 실제가 1이면 정답, 0이면 틀린 것임
    merged_df['Check'] = merged_df['result_is_failure'].apply(
        lambda x: 'O (정답)' if x == 1 else 'X (오진)'
    )

    # 보기 좋게 컬럼 이름 변경
    merged_df.columns = ['에피소드_번호', 'AI_예측확률', '실제_결과(1=고장)', '판정']

    # 4. 저장 및 출력
    merged_df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

    print(f"\n✅ 매칭 완료! 저장된 파일: {OUTPUT_PATH}")
    print("\n[결과 미리보기]")
    print(merged_df.head(10)) # 상위 10개 출력

    # 5. 간단 요약
    correct = len(merged_df[merged_df['실제_결과(1=고장)'] == 1])
    total = len(merged_df)
    print(f"\n📊 요약: AI가 위험하다고 한 {total}개 중 {correct}개가 실제로 고장남.")
    print(f"   -> 적중률: {(correct/total)*100:.1f}%")

if __name__ == "__main__":
    main()

📖 데이터 읽는 중...

✅ 매칭 완료! 저장된 파일: /content/drive/MyDrive/Robot_Analysis/simple_match_result.csv

[결과 미리보기]
   에피소드_번호   AI_예측확률  실제_결과(1=고장)      판정
0        5  0.988862            1  O (정답)
1        6  0.989561            1  O (정답)
2        7  0.988293            1  O (정답)
3        8  0.912907            1  O (정답)
4       11  0.989370            1  O (정답)
5       12  0.808132            1  O (정답)
6       21  0.962442            1  O (정답)
7       24  0.989074            1  O (정답)
8       29  0.988117            1  O (정답)
9       30  0.964231            1  O (정답)

📊 요약: AI가 위험하다고 한 657개 중 575개가 실제로 고장남.
   -> 적중률: 87.5%


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import math
import os
import sys
from google.colab import drive

# ==========================================
# 1. 구글 드라이브 연결
# ==========================================
if not os.path.exists('/content/drive'):
    print("📂 구글 드라이브 연결 중...")
    drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/Robot_Analysis'

# 파일 경로
X_TEST_PATH = os.path.join(BASE_DIR, 'X_test.npy')
MODEL_PATH = os.path.join(BASE_DIR, 'transformer_model.pt')
OUTPUT_PATH = os.path.join(BASE_DIR, 'danger_max_duration_summary.csv') # 최종 결과 파일명

# 원본 데이터 (Min/Max 기준용)
TRAIN_STEPS_PATH = os.path.join(BASE_DIR, 'train_steps.csv')
TRAIN_SUMMARY_PATH = os.path.join(BASE_DIR, 'train_summary.csv')
VAL_STEPS_PATH = os.path.join(BASE_DIR, 'val_steps.csv')
VAL_SUMMARY_PATH = os.path.join(BASE_DIR, 'val_summary.csv')

# 설정
INPUT_DIM = 9
SEQ_LEN = 2400
D_MODEL = 64
NHEAD = 4
NUM_LAYERS = 3
DIM_FEEDFORWARD = 128
DROPOUT = 0.1
NUM_CLASSES = 2
AI_THRESHOLD = 0.58

# 컬럼 이름 (CSV 헤더와 일치해야 함)
FEATURE_COLS = [
    'speed', 'dist_to_wall', 'drag_force_N',
    'mass_kg', 'friction_cond', 'air_density', 'trigger_dist_m', 'brake_torque', 'init_speed_cmd'
]
STATIC_COLS = [
    'episode', 'mass_kg', 'friction_cond', 'air_density', 'brake_torque', 'init_speed_cmd'
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. 모델 클래스 정의
# ==========================================
class BasePredictionModel(nn.Module):
    def forward(self, x): raise NotImplementedError

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

class TransformerModel(BasePredictionModel):
    def __init__(self, input_dim, seq_len, num_classes=2, d_model=64, nhead=4, num_layers=3, dim_feedforward=128, dropout=0.1, use_patch=False, patch_len=10, patch_stride=5):
        super().__init__()
        self.input_dim = input_dim
        self.seq_len = seq_len
        self.use_patch = use_patch
        self.patch_len = patch_len
        self.patch_stride = patch_stride

        if use_patch:
            token_dim = patch_len * input_dim
            if seq_len < patch_len:
                num_tokens = 1
            else:
                num_tokens = 1 + (seq_len - patch_len) // patch_stride
            self.num_tokens = num_tokens
        else:
            token_dim = input_dim
            self.num_tokens = seq_len

        self.input_proj = nn.Linear(token_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model=d_model, max_len=self.num_tokens)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.dropout = nn.Dropout(dropout)
        self.cls_head = nn.Linear(d_model, num_classes)
        self.seq_head = nn.Linear(d_model, 1)

    def forward(self, x, return_sequence=True):
        tokens = x
        tokens = self.input_proj(tokens)
        tokens = self.pos_encoder(tokens)
        h = self.transformer_encoder(tokens)

        if return_sequence:
            logits_seq = self.seq_head(h)
            probs_seq = torch.sigmoid(logits_seq)
            return probs_seq
        else:
            h_mean = h.mean(dim=1)
            h_mean = self.dropout(h_mean)
            logits = self.cls_head(h_mean)
            return logits

# ==========================================
# 3. 데이터 로드 및 Min/Max 계산
# ==========================================
def load_and_merge(steps_path, summary_path):
    if not os.path.exists(steps_path) or not os.path.exists(summary_path):
        return None
    df_steps = pd.read_csv(steps_path)
    df_summary = pd.read_csv(summary_path)
    df_merged = pd.merge(df_steps, df_summary[STATIC_COLS], on='episode', how='left')
    return df_merged

def calculate_global_minmax():
    print("⚖️ 정규화 기준값 계산 중 (Train + Val 병합)...")

    df_train = load_and_merge(TRAIN_STEPS_PATH, TRAIN_SUMMARY_PATH)
    if df_train is None:
        print("❌ Train 파일이 없습니다. 경로를 확인하세요.")
        return None, None

    df_val = load_and_merge(VAL_STEPS_PATH, VAL_SUMMARY_PATH)

    if df_val is not None:
        df_all = pd.concat([df_train, df_val], axis=0, ignore_index=True)
    else:
        df_all = df_train

    try:
        target_data = df_all[FEATURE_COLS]
        min_vals = target_data.min().values
        max_vals = target_data.max().values
    except KeyError as e:
        print(f"❌ 컬럼 이름 오류: {e}")
        print(f"   CSV 컬럼: {list(df_all.columns)}")
        return None, None

    print("\n📊 [MinMax 기준값]")
    for i, col in enumerate(FEATURE_COLS):
        print(f"   - {col}: Min={min_vals[i]:.4f}, Max={max_vals[i]:.4f}")

    return min_vals, max_vals

# ==========================================
# 4. 메인 실행 함수
# ==========================================
def main():
    # (1) 기준값 계산
    data_min, data_max = calculate_global_minmax()
    if data_min is None: return

    data_range = data_max - data_min
    data_range[data_range == 0] = 1.0

    # (2) 입력 데이터 로드
    if not os.path.exists(X_TEST_PATH):
        print(f"❌ npy 파일 없음: {X_TEST_PATH}")
        return

    X_test_np = np.load(X_TEST_PATH)
    print(f"\n📂 입력 데이터(npy) 로드 완료: {X_test_np.shape}")
    X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).to(device)

    # (3) 모델 예측
    print("\n🤖 Transformer 모델 로드 및 예측...")
    model = TransformerModel(input_dim=INPUT_DIM, seq_len=SEQ_LEN, num_classes=NUM_CLASSES, d_model=D_MODEL, nhead=NHEAD, num_layers=NUM_LAYERS, use_patch=False).to(device)

    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.eval()
    else:
        print("❌ 모델 파일 없음")
        return

    print("🔮 위험도 예측 중...")
    BATCH_SIZE = 32
    all_preds = []

    with torch.no_grad():
        num_samples = X_test_tensor.shape[0]
        for i in range(0, num_samples, BATCH_SIZE):
            batch_x = X_test_tensor[i : i+BATCH_SIZE]
            batch_pred = model(batch_x, return_sequence=True)
            all_preds.append(batch_pred.cpu().numpy())

    predictions = np.concatenate(all_preds, axis=0).squeeze(-1)

    # (4) ★ 핵심: 최장 위험 구간의 시작점 찾기 ★
    print(f"\n🔍 에피소드별 '가장 길게 지속된' 위험 구간 추출 중...")

    extracted_rows = []

    for ep_idx, episode_probs in enumerate(predictions):
        # 위험 마스크 (True/False)
        is_danger = episode_probs >= AI_THRESHOLD

        # 상태 변화 지점 찾기 (Start/End)
        padded = np.concatenate(([False], is_danger, [False]))
        diff = np.diff(padded.astype(int))

        starts = np.where(diff == 1)[0]
        ends = np.where(diff == -1)[0]

        # 위험 구간이 하나라도 있다면
        if len(starts) > 0:
            # 각 구간의 길이(Duration) 계산
            durations = ends - starts

            # 가장 긴 구간의 인덱스 찾기 (Argmax)
            longest_idx = np.argmax(durations)

            # 그 구간의 정보 가져오기
            best_start = starts[longest_idx]
            best_duration = durations[longest_idx]
            best_prob = np.mean(episode_probs[best_start : ends[longest_idx]]) # 구간 평균 확률

            # 시작 시점의 물리값 복원
            norm_vals = X_test_np[ep_idx, best_start, :]
            real_vals = norm_vals * data_range + data_min

            # [에피소드, 시작시간, 지속시간, 평균확률, ...물리변수들...]
            row = [ep_idx, best_start, best_duration, best_prob] + real_vals.tolist()
            extracted_rows.append(row)

    # (5) 저장
    if len(extracted_rows) > 0:
        result_df = pd.DataFrame(extracted_rows, columns=["Episode_ID", "Start_Time", "Max_Duration", "Avg_Prob"] + FEATURE_COLS)

        result_df.to_csv(OUTPUT_PATH, index=False)
        print(f"\n✅ 분석 완료! 총 {len(result_df)}건 (에피소드당 1개) 저장됨.")
        print(f"💾 파일 경로: {OUTPUT_PATH}")
        print(result_df.head())

        # (6) 자동 분석 (Risk Score)
        print("\n" + "="*60)
        print("📊 [자동 분석] 최장 위험 구간 시작점의 통계")
        print("="*60)

        dists = result_df['dist_to_wall'].replace(0, 0.01)
        frictions = result_df['friction_cond'].replace(0, 0.01)

        # Risk = Speed^2 / (Friction * Dist)
        risk_scores = (result_df['speed'] ** 2) / (frictions * dists)

        th_10 = np.percentile(risk_scores, 10)

        print(f"평균 Risk Score: {risk_scores.mean():.2f}")
        print(f"🚨 추천 경보 임계값 (하위 10%): {th_10:.2f}")

    else:
        print("\n⚠️ 0.58을 넘는 위험 구간이 전혀 없습니다.")

if __name__ == "__main__":
    main()

⚖️ 정규화 기준값 계산 중 (Train + Val 병합)...

📊 [MinMax 기준값]
   - speed: Min=0.0053, Max=2.5404
   - dist_to_wall: Min=0.0000, Max=6.3000
   - drag_force_N: Min=0.0000, Max=0.1724
   - mass_kg: Min=4.0002, Max=7.9996
   - friction_cond: Min=0.1000, Max=1.0000
   - air_density: Min=1.1500, Max=1.3500
   - trigger_dist_m: Min=0.0500, Max=0.3000
   - brake_torque: Min=2.0000, Max=2.0000
   - init_speed_cmd: Min=60.0017, Max=79.9995

📂 입력 데이터(npy) 로드 완료: (2033, 2400, 9)

🤖 Transformer 모델 로드 및 예측...
🔮 위험도 예측 중...

🔍 에피소드별 '가장 길게 지속된' 위험 구간 추출 중...

✅ 분석 완료! 총 657건 (에피소드당 1개) 저장됨.
💾 파일 경로: /content/drive/MyDrive/Robot_Analysis/danger_max_duration_summary.csv
   Episode_ID  Start_Time  Max_Duration  Avg_Prob     speed  dist_to_wall  \
0           5           0          1726  0.988862  0.416270      6.300002   
1           6           0          2152  0.989561  0.416270      6.300001   
2           7           0          1664  0.988293  0.416270      6.300000   
3           8           0          2400 